In [8]:
# Advanced deep-learning best practices: Introduction to the functional API

from tensorflow.keras import Input, layers

import warnings
warnings.filterwarnings('ignore')

from pandas.plotting import scatter_matrix

# Preparing higher-resolution data generator for the jena datset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential

from tensorflow.keras import layers 

from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras import applications, layers, Input


%matplotlib inline
import numpy as np
import datetime
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from sklearn.datasets import make_blobs
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn import utils

from pylab import mpl, plt
import datetime
import tensorflow as tf
from tensorflow import keras
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [9]:
# Reweighting a probality distribution to a different temprature 

import numpy as np 

# orginal distribution is a 1D np array of prebability values that must sum to 1.
# temperature is a factor quantifying the entropy is a factor quantifying the entropy of the output distribution.
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution)/ temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)

# Returns a reweighted version of the orginal distrbution. The sum of the distribution may no longer be 1,
    # so divide it by its sum ot ontrain the new distribution.

In [16]:
# Implementing character-level LSTM text generation

path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()

print('Corpus length:', len(text))

Corpus length: 600901


In [28]:
# Vectorizing sequences of charecters 

maxlen = 60 

step = 3 

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of seqiences:', len(sentences))

chars = sorted(list(set(text)))

print('Unique characters:', len(chars))

char_indeces = dict((char, chars.index(char)) for char in chars)

print("Vectorization...")

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i , sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indeces[char]] =1
    y[i, char_indeces[next_chars[i]]] = 1

Number of seqiences: 200281
Unique characters: 59
Vectorization...


In [38]:
"""
Building The NetWork:
This network is a single LSTM layer followed by a Dense classifier and softmax over all possible charecters.
Note: 1D convnets also have proven extremly successful at this task in rencent times.
"""

import tensorflow

from tensorflow.keras import models


char_preModel = tensorflow.keras.models.Sequential()

char_preModel.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))

char_preModel.add(layers.Dense(len(chars), activation='softmax'))

# Used categorical_crossentropy to deal with one-hot encoded targets.

# Model compilation configuration 

optimizer = keras.optimizers.RMSprop(lr=0.01)

char_preModel.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [40]:
# Function to sample the next character given the model's predictions 

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [41]:
"""
Finally, the following loop repeatedly trains and generates text.
You begin generating text using a range of different temeparatures after every epoch. 
This allows you to see how the genarated text evolves as the model begins to convege,
as well as the impact of temperature in the sampling strategy.
"""


'\nFinally, the following loop repeatedly trains and generates text.\nYou begin generating text using a range of different temeparatures after every epoch. \nThis allows you to see how the genarated text evolves as the model begins to convege,\nas well as the impact of temperature in the sampling strategy.\n'

In [ ]:
# Text-generation loop

import random 
import sys

for epoch in range(1, 60):
    print('epoch',epoch)
    char_preModel.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

epoch 1
Train on 200281 samples
200281/200281 [==============================] - 92s 458us/sample - loss: 1.6346
--- Generating with seed: "ain, "stronger, more evil, and more profound; also more
beau"
epoch 2
Train on 200281 samples
200281/200281 [==============================] - 91s 454us/sample - loss: 1.5443
--- Generating with seed: "hand-dilation,
this incense-fuming exaltation? is ours this "
epoch 3
Train on 200281 samples
200281/200281 [==============================] - 92s 457us/sample - loss: 1.4985
--- Generating with seed: " of
independence.

42. a new order of philosophers is appear"
epoch 4
Train on 200281 samples
200281/200281 [==============================] - 91s 456us/sample - loss: 1.4684
--- Generating with seed: "phlegm in the former and
with hard skulls in the latter--not"
epoch 5
Train on 200281 samples
200281/200281 [==============================] - 91s 455us/sample - loss: 1.4479
--- Generating with seed: "endowed with
intellectual goods and privileges, are 

In [ ]:
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:' temperature)
    sys.stdout.write(generated_text)
    
    
for i in range(400):
    sampled = np.zeors((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1 
        
    preds = char_preModel.predict(sampled, verbore=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    
    generated_text += next_char
    generated_text = generated_text[1:]
    
    sys.stdout.write(next_char)